# Imports

In [1]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal


# Set Paths

In [2]:
# Define the project path and a list of 5 ABF file names
project_path = 'C:\\Users\\ikmor\\OneDrive\\Documents\\SNR AND OBR pilot study\\'
abf_files = [
    f'{project_path}PTX8R3 ACCIN.abf',
    f'{project_path}PTX7L2 ACCOM.abf',
    f'{project_path}PTX09R5 ACCOM3.abf',
    f'{project_path}PTX6L2 ACCOM.abf'
]


In [3]:
# Initialize a list to store total power sums for each file
file_total_powers = []

# Initialize a 2D list to store power for each sweep (current injection) across all files
sweep_powers = [[] for _ in range(10)]  # Changed to 10 sweeps

# Initialize a list to store average total power for each file
file_average_powers = []

LJP = 13  # Liquid Junction Potential in mV

# Loop through each ABF file
for abf_file in abf_files:
    # Load the ABF file
    abf = pyabf.ABF(abf_file)
    
    # Define sweep numbers and initialize total power sum for this file
    sweep_numbers = range(10)  # Changed to range(10) for 10 sweeps
    total_power_sum = 0
    sweep_power_sum = 0

    # Loop through each sweep
    for sweep in sweep_numbers:
        abf.setSweep(sweep, channel=0)
        
        # Define the time segment for analysis (in seconds)
        start_time = 3  # Start time in seconds
        end_time = 9    # End time in seconds

        # Extract voltage (sweepY) and time (sweepX)
        voltage = abf.sweepY - LJP  # Voltage in mV
        times = abf.sweepX    # Time in seconds

        # Extract current command waveform (sweepC)
        current_command = abf.sweepC  # Current in pA

        # Find indices corresponding to the desired time segment
        start_index = np.argmin(np.abs(times - start_time))
        end_index = np.argmin(np.abs(times - end_time))

        # Extract the specific portion of the data for analysis
        segment_voltage = voltage[start_index:end_index]
        segment_current = current_command[start_index:end_index]

        # Convert current from pA to A and voltage from mV to V for power calculation
        segment_current_A = segment_current * 1e-12
        segment_voltage_V = segment_voltage * 1e-3

        # Calculate instantaneous power: P = V * I
        instantaneous_power = segment_voltage_V * segment_current_A

        # Calculate the Power Spectral Density (PSD) using Welch's method with zero padding
        fs = abf.dataRate  # Sampling frequency from ABF file
        segment_length_ms = 20  # Optimal segment length in milliseconds
        nperseg = int(fs * segment_length_ms / 1000)
        noverlap = nperseg // 2  # 50% overlap

        # Ensure nperseg is an even number for efficient FFT
        nperseg += nperseg % 2

        # Zero padding
        nfft = 320000  # Increase FFT size such that frequency resolution is 0.05Hz

        # Apply Welch's method with zero padding
        f, Pxx = signal.welch(instantaneous_power, fs=fs, window='hann', nperseg=nperseg,
                              noverlap=noverlap, nfft=nfft, scaling='density')

        # Calculate total power excluding frequencies below 0.1 Hz
        freq_mask = f >= 0.1
        average_power_above_0_1Hz = np.trapz(Pxx[freq_mask], f[freq_mask])  # Integrate PSD above 0.1 Hz

        # Calculate the duration of the sweep and total power for this sweep
        sweep_duration = end_time - start_time
        total_power_above_0_1Hz = average_power_above_0_1Hz * sweep_duration * 1e24  # Convert to yoctoWatt

        # Add the total power for this sweep to the sum for this file
        total_power_sum += total_power_above_0_1Hz
        sweep_power_sum += total_power_above_0_1Hz

        # Store the power for this sweep in the corresponding list
        sweep_powers[sweep].append(total_power_above_0_1Hz)

    # Append the total power sum for this file to the list and print it
    file_total_powers.append(total_power_sum)
    print(f"\nSum of total power across all sweeps for file {abf_file}: {total_power_sum:.6e} yoctoWatt")

    # Calculate and append the average total power for this file
    average_power = sweep_power_sum / len(sweep_numbers)
    file_average_powers.append(average_power)
    print(f"Average total power across all sweeps for file {abf_file}: {average_power:.6e} yoctoWatt")

# Calculate and print the average of summed powers across all files
average_total_power = np.mean(file_total_powers)
print(f"\nAverage of summed power across all files: {average_total_power:.6e} yoctoWatt")

# Calculate and print the average of average powers across all files
average_of_average_powers = np.mean(file_average_powers)
print(f"\nAverage of average powers across all files: {average_of_average_powers:.6e} yoctoWatt")

# Calculate the average of total power for each sweep across files
average_sweep_powers = [np.mean(powers) for powers in sweep_powers]

# Calculate and print the variance of these averages
variance_across_injections = np.var(average_sweep_powers)
print(f"\nVariance of average summed power across current injections: {variance_across_injections:.6e} yoctoWatt^2")

# Calculate and print the variance of power across the files for each current injection
print("\nVariance of power across files for each current injection:")
for sweep, powers in enumerate(sweep_powers):
    variance = np.var(powers)
    print(f"Current injection {sweep}: {variance:.6e} yoctoWatt^2")

# Calculate and print the average of intertrial variability power
average_intertrial_variability = np.mean([np.var(powers) for powers in sweep_powers])
print(f"\nAverage of intertrial variability power: {average_intertrial_variability:.6e} yoctoWatt^2")


Sum of total power across all sweeps for file C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\PTX8R3 ACCIN.abf: 6.232218e+01 yoctoWatt
Average total power across all sweeps for file C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\PTX8R3 ACCIN.abf: 6.232218e+00 yoctoWatt

Sum of total power across all sweeps for file C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\PTX7L2 ACCOM.abf: 3.401334e+01 yoctoWatt
Average total power across all sweeps for file C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\PTX7L2 ACCOM.abf: 3.401334e+00 yoctoWatt

Sum of total power across all sweeps for file C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\PTX09R5 ACCOM3.abf: 4.368657e+01 yoctoWatt
Average total power across all sweeps for file C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\PTX09R5 ACCOM3.abf: 4.368657e+00 yoctoWatt

Sum of total power across all sweeps for file C:\Users\ikmor\OneDrive\Documents\SNR AND OBR pilot study\PTX6L2 ACCOM.ab